# Measuring province-level GDP using nighttime lights, net primary productivity, and land cover

## Install Packages

In [1]:
import pandas as pd
# import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandasgui import show

## Preliminaries

In [2]:
# Set path
path = "/Users/Jesson Pagaduan/Google Drive/Geospatial_Project/"

## Non-agri GDP by province from NTL

In [80]:
# Export NTL series at the province level
df_ntl_prov = pd.read_csv(path + "data/scratch/NTL_2000-2018_PHL_province_21Apr.csv", index_col=False,
                          usecols=['YEAR', 'NAME_1', 'ZONE_CODE', 'COUNT', 'SUM'], 
                          parse_dates=['YEAR'])
df_ntl_prov.columns = [i.lower() for i in df_ntl_prov.columns]

# Create a region column
df_ntl_prov['region'] = ''

# Create region placeholders
ncr = ['Metropolitan Manila']
car = ['Abra', 'Apayao', 'Benguet', 'Ifugao', 'Kalinga', 'Mountain Province']
region1 = ['Ilocos Norte', 'Ilocos Sur', 'La Union', 'Pangasinan']
region2 = ['Batanes', 'Cagayan', 'Isabela', 'Nueva Vizcaya', 'Quirino']
region3 = ['Aurora', 'Bataan', 'Bulacan', 'Nueva Ecija', 'Pampanga', 'Tarlac', 'Zambales']
region4a = ['Batangas', 'Cavite', 'Laguna', 'Quezon', 'Rizal']
region4b = ['Marinduque', 'Occidental Mindoro', 'Oriental Mindoro', 'Palawan', 'Romblon']
region5 = ['Albay', 'Camarines Norte', 'Camarines Sur', 'Catanduanes', 'Masbate', 'Sorsogon']
region6 = ['Aklan', 'Antique', 'Capiz', 'Guimaras', 'Iloilo', 'Negros Occidental']
region7 = ['Bohol', 'Cebu', 'Negros Oriental', 'Siquijor']
region8 = ['Biliran', 'Eastern Samar', 'Leyte', 'Northern Samar', 'Samar', 'Southern Leyte']
region9 = ['Zamboanga del Norte', 'Zamboanga del Sur', 'Zamboanga Sibugay']
region10 = ['Bukidnon', 'Camiguin', 'Lanao del Norte', 'Misamis Occidental', 'Misamis Oriental']
region11 = ['Compostela Valley', 'Davao del Norte', 'Davao del Sur', 'Davao Oriental']
region12 = ['North Cotabato', 'Sarangani', 'South Cotabato', 'Sultan Kudarat']
region13 = ['Agusan del Norte', 'Agusan del Sur', 'Dinagat Islands', 'Surigao del Norte', 'Surigao del Sur']
armm = ['Basilan', 'Lanao del Sur', 'Maguindanao', 'Sulu', 'Tawi-Tawi']

# Assign provinces to regions
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(ncr), 'region'] = 'Metropolitan Manila'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(car), 'region'] = 'Cordillera Administrative Region (CAR)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region1), 'region'] = 'Ilocos Region (Region I)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region2), 'region'] = 'Cagayan Valley (Region II)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region3), 'region'] = 'Central Luzon (Region III)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region4a), 'region'] = 'CALABARZON (Region IV-A)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region4b), 'region'] = 'MIMAROPA (Region IV-B)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region5), 'region'] = 'Bicol Region (Region V)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region6), 'region'] = 'Western Visayas (Region VI)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region7), 'region'] = 'Central Visayas (Region VII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region8), 'region'] = 'Eastern Visayas (Region VIII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region9), 'region'] = 'Zamboanga Peninsula (Region IX)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region10), 'region'] = 'Northern Mindanao (Region X)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region11), 'region'] = 'Davao Region (Region XI)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region12), 'region'] = 'SOCCSKSARGEN (Region XII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(region13), 'region'] = 'Caraga (Region XIII)'
df_ntl_prov.loc[df_ntl_prov['name_1'].isin(armm), 'region'] = 'Autonomous Region of Muslim Mindanao (ARMM)'

# Keep relevant columns
df_ntl_prov = df_ntl_prov[['year', 'region', 'name_1', 'zone_code', 'count', 'sum']]

# Export NTL series at the region level
df_ntl_reg = pd.read_csv(path + "data/scratch/data.csv", index_col=False,
                         usecols=['YEAR', 'REGION', 'ZONE_CODE', 'NTL_COUNT', 'NTL_SUM'], 
                         parse_dates=['YEAR'])
df_ntl_reg.columns = [i.lower() for i in df_ntl_reg.columns]

# Merge the two dataframes
df_ntl_prov = df_ntl_reg.set_index(['region', 'year']).join(df_ntl_prov.set_index(['region', 'year']),
                                                            rsuffix='_prov')
df_ntl_prov.columns = ['reg_id', 'reg_count_lit', 'reg_ntl', 'province', 'prov_id', 'prov_count_lit',
                       'prov_ntl']

# Calculate province shares
df_ntl_prov = df_ntl_prov.assign(prov_ntl_share = lambda x: x.prov_ntl / x.reg_ntl)

### Using predicted regional GDP

In [15]:
# Export predicted GDP
df_predictions = pd.read_excel(path + "gdp_predictions.xlsx", index_col=False,
                               usecols=['region', 'year', 'period', 'zone_code', 'ind_ntl_top13_p1', 
                                        'svc_ntl_top13_p1', 'non_agr_ntl_top13_p1', 'ind_ntl_top13_p2',
                                        'svc_ntl_top13_p2', 'non_agr_ntl_top13_p2'], 
                               parse_dates=['year'])
df_predictions.columns = [i.lower() for i in df_predictions.columns]

# Join the two dataframes
df_ntl_prov = df_ntl_prov.join(df_predictions.set_index(['region', 'year']), rsuffix='_pred')

# Calculate province-level non-agri GDP
df_ntl_prov = df_ntl_prov.assign(log_ind_p1 = lambda x: x.ind_ntl_top13_p1 * x.prov_ntl_share,
                                 log_ind_p2 = lambda x: x.ind_ntl_top13_p2 * x.prov_ntl_share,
                                 log_svc_p1 = lambda x: x.svc_ntl_top13_p1 * x.prov_ntl_share,
                                 log_svc_p2 = lambda x: x.svc_ntl_top13_p2 * x.prov_ntl_share,
                                 log_non_agr_p1 = lambda x: x.non_agr_ntl_top13_p1 * x.prov_ntl_share,
                                 log_non_agr_p2 = lambda x: x.non_agr_ntl_top13_p2 * x.prov_ntl_share)

# Calculate province-level non-agri GDP growth rates
df_ntl_prov_gr = df_ntl_prov[['province', 'prov_id', 'log_ind_p1', 'log_ind_p2', 'log_svc_p1', 
                              'log_svc_p2', 'log_non_agr_p1', 
                              'log_non_agr_p2']].groupby(by=['province', 'prov_id']).diff() * 100


df_ntl_prov = df_ntl_prov.assign(log_ind_p1_gr = df_ntl_prov[['province', 'log_ind_p1']].groupby(by='province').diff() * 100,
                                 log_ind_p2_gr = df_ntl_prov[['province', 'log_ind_p2']].groupby(by='province').diff() * 100,
                                 log_svc_p1_gr = df_ntl_prov[['province', 'log_svc_p1']].groupby(by='province').diff() * 100,
                                 log_svc_p2_gr = df_ntl_prov[['province', 'log_svc_p2']].groupby(by='province').diff() * 100,
                                 log_non_agr_p1_gr = df_ntl_prov[['province', 'log_non_agr_p1']].groupby(by='province').diff() * 100,
                                 log_non_agr_p2_gr = df_ntl_prov[['province', 'log_non_agr_p2']].groupby(by='province').diff() * 100)

# Export to excel
# df_ntl_prov.reset_index().to_excel('province-gdp.xlsx', index=False)

# Calculate long differences
df_longdiff = df_ntl_prov.reset_index().pivot(index=['province', 'prov_id'], columns=['year'],
                                              values=['log_non_agr_p1', 'log_non_agr_p2']).reset_index()
df_longdiff = df_longdiff.iloc[:, [0, 1, 2, 10, 30, 39]]
df_longdiff.columns = ['province', 'prov_id', 'log_non_agr_gdp_2000', 'log_non_agr_gdp_2008',
                       'log_non_agr_gdp_2009', 'log_non_agr_gdp_2018']
df_longdiff = df_longdiff.assign(ld_non_agr_gdp_p1 = lambda x: 100 * (x.log_non_agr_gdp_2008 - x.log_non_agr_gdp_2000),
                                 ld_non_agr_gdp_p2 = lambda x: 100 * (x.log_non_agr_gdp_2018 - x.log_non_agr_gdp_2009))
# df_longdiff.to_csv('province-gdp-longdiff.csv', index=False)

# Calculate average GDP growth
df_ntl_avg_gr = df_ntl_prov[['province', 'log_non_agr_p1_gr']].groupby(by='province').mean().join(df_ntl_prov[['province', 'log_non_agr_p2_gr']].groupby(by='province').mean())
df_ntl_avg_gr.reset_index(inplace=True)
# df_ntl_avg_gr.to_csv('province-gdp-avg-gr.csv', index=False)

# Create year-specific averages
df_ntl_prov.reset_index(inplace=True)
df_ntl_prov_2000 = df_ntl_prov[df_ntl_prov['year']=='2000-01-01']
df_ntl_prov_2000 = df_ntl_prov_2000[['province', 'prov_id', 'log_non_agr_p1', 'log_ind_p1', 'log_svc_p1']]
# df_ntl_prov_2000.to_csv('province-gdp-2000.csv', index=False)
df_ntl_prov_2018 = df_ntl_prov[df_ntl_prov['year']=='2018-01-01']
df_ntl_prov_2018 = df_ntl_prov_2018[['province', 'prov_id', 'log_non_agr_p2', 'log_ind_p2', 'log_svc_p2']]
# df_ntl_prov_2018.to_csv('province-gdp-2018.csv', index=False)
df_ntl_prov_2009 = df_ntl_prov[df_ntl_prov['year']=='2009-01-01']
df_ntl_prov_2009 = df_ntl_prov_2009[['province', 'prov_id', 'log_non_agr_p2', 'log_ind_p2', 'log_svc_p2']]
# df_ntl_prov_2009.to_csv('province-gdp-2009.csv', index=False)
df_ntl_prov_2008 = df_ntl_prov[df_ntl_prov['year']=='2008-01-01']
df_ntl_prov_2008 = df_ntl_prov_2008[['province', 'prov_id', 'log_non_agr_p1', 'log_ind_p1', 'log_svc_p1']]
# df_ntl_prov_2008.to_csv('province-gdp-2008.csv', index=False)

### Using observed regional GDP

In [81]:
# Export data on observed regional GDP
df_observed_nonagr = pd.read_csv(path + "data/scratch/data.csv", index_col=False,
                                 usecols=['REGION', 'YEAR', 'ZONE_CODE', 'GDP_IND', 'GDP_SVC'], 
                                 parse_dates=['YEAR'])
df_observed_nonagr.columns = [i.lower() for i in df_observed_nonagr.columns]

# Calculate non-agri GDP (industry + services)
df_observed_nonagr = df_observed_nonagr.assign(gdp_non_agr = lambda x: x.gdp_ind + x.gdp_svc)
df_observed_nonagr.drop(columns=['gdp_ind', 'gdp_svc'], inplace=True)

# Join two dataframes
df_gdp_nonagr_prov = df_ntl_prov.join(df_observed_nonagr.set_index(['region', 'year']))

# Calculate province-level non-agri GDP based on NTL shares
df_gdp_nonagr_prov = df_gdp_nonagr_prov.assign(gdp_nonagr_prov = lambda x: x.gdp_non_agr * x.prov_ntl_share)

## Agriculture GDP by province from Land Cover

In [83]:
# Export province-level land cover series
df_lc_prov = pd.read_excel(path + "data/scratch/LC_2001-2019_PHL_province.xlsx", index_col=False,
                           usecols=['YEAR', 'NAME_1', 'OBJECTID', 'VALUE_12'], 
                           parse_dates=['YEAR'])
df_lc_prov.columns = [i.lower() for i in df_lc_prov.columns]

# Create a region column
df_lc_prov['region'] = ''

# Assign provinces to respective regions
df_lc_prov.loc[df_lc_prov['name_1'].isin(ncr), 'region'] = 'Metropolitan Manila'
df_lc_prov.loc[df_lc_prov['name_1'].isin(car), 'region'] = 'Cordillera Administrative Region (CAR)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region1), 'region'] = 'Ilocos Region (Region I)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region2), 'region'] = 'Cagayan Valley (Region II)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region3), 'region'] = 'Central Luzon (Region III)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region4a), 'region'] = 'CALABARZON (Region IV-A)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region4b), 'region'] = 'MIMAROPA (Region IV-B)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region5), 'region'] = 'Bicol Region (Region V)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region6), 'region'] = 'Western Visayas (Region VI)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region7), 'region'] = 'Central Visayas (Region VII)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region8), 'region'] = 'Eastern Visayas (Region VIII)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region9), 'region'] = 'Zamboanga Peninsula (Region IX)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region10), 'region'] = 'Northern Mindanao (Region X)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region11), 'region'] = 'Davao Region (Region XI)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region12), 'region'] = 'SOCCSKSARGEN (Region XII)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(region13), 'region'] = 'Caraga (Region XIII)'
df_lc_prov.loc[df_lc_prov['name_1'].isin(armm), 'region'] = 'Autonomous Region of Muslim Mindanao (ARMM)'

# Keep relevant columns only
df_lc_prov = df_lc_prov[['year', 'region', 'name_1', 'objectid', 'value_12']]

# Export region-level land cover series
df_lc_reg = pd.read_csv(path + "data/scratch/data.csv", index_col=False,
                        usecols=['YEAR', 'REGION', 'ZONE_CODE', 'LC_C12'], 
                        parse_dates=['YEAR'])
df_lc_reg.columns = [i.lower() for i in df_lc_reg.columns]

# Join the two dataframes
df_lc_prov = df_lc_reg.set_index(['region', 'year']).join(df_lc_prov.set_index(['region', 'year']),
                                                          rsuffix='_prov')

# Rename columns
df_lc_prov.columns = ['reg_code', 'croplands_reg', 'province', 'prov_code', 'croplands_prov']

# Calculate province's croplands share
df_lc_prov = df_lc_prov.assign(prov_lc_share = lambda x: x.croplands_prov / x.croplands_reg)

### Using predicted regional GDP

In [71]:
# Export predicted regional agri-GDP
df_agr_predictions = pd.read_excel(path + "ntl_predictions_26Apr.xls", index_col=False,
                               usecols=['region', 'year', 'period', 'zone_code', 
                                        'agr_c1012_top12c12_p1', 'agr_c1012_top12c12_p2'], 
                               parse_dates=['year'])
df_agr_predictions.columns = [i.lower() for i in df_agr_predictions.columns]

# Join the two dataframes
df_lc_prov = df_lc_prov.join(df_agr_predictions.set_index(['region', 'year']), rsuffix='_pred')

# Calculate province-level agri GDP
df_lc_prov = df_lc_prov.assign(log_agr_p1 = lambda x: x.agr_c1012_top12c12_p1 * x.prov_lc_share,
                               log_agr_p2 = lambda x: x.agr_c1012_top12c12_p2 * x.prov_lc_share)

# Drop obs for 2000 because there is land cover data
df_lc_prov.drop(index=['2000-01-01'], level=1, inplace=True)

# Calculate growth rates
df_lc_prov_gr = df_lc_prov.reset_index().set_index(['province', 'year'])[['prov_code', 'log_agr_p1', 
                                                                          'log_agr_p2']].groupby('prov_code').diff() * 100
# df_lc_prov_gr.reset_index().to_excel('province-agr-gr.xlsx', index=False)
# df_lc_prov_gr.reset_index().groupby('province').mean().reset_index().to_csv('province-agr-gr-avg.csv', index=False)

### Using observed regional GDP

In [85]:
# Export data on observed regional GDP
df_observed_agr = pd.read_csv(path + "data/scratch/data.csv", index_col=False,
                              usecols=['REGION', 'YEAR', 'ZONE_CODE', 'GDP_AGR'], 
                              parse_dates=['YEAR'])
df_observed_agr.columns = [i.lower() for i in df_observed_agr.columns]

# Join two dataframes
df_gdp_agr_prov = df_lc_prov.join(df_observed_agr.set_index(['region', 'year']))

# Calculate province-level non-agri GDP based on NTL shares
df_gdp_agr_prov = df_gdp_agr_prov.assign(gdp_agr_prov = lambda x: x.gdp_agr * x.prov_lc_share)

## Province-level GDP from NTL and Land Cover

In [137]:
# Merge non-agri and agri dataframes
df_gdp_prov = df_gdp_nonagr_prov.reset_index().set_index(['province', 'year']).join(df_gdp_agr_prov.reset_index()[df_gdp_agr_prov.reset_index()['year'] > '2000-01-01'].set_index(['province', 'year']), rsuffix='_r')
df_gdp_prov = df_gdp_prov[['region', 'reg_id', 'prov_id', 'gdp_nonagr_prov', 'gdp_agr_prov']]

# Calculate total province GDP (non-agri + agri)
df_gdp_prov = df_gdp_prov.assign(gdp = lambda x: x.gdp_nonagr_prov + x.gdp_agr_prov)

# Export to Excel
df_gdp_prov.reset_index().to_excel('province-gdp.xlsx', index=False)